In [ ]:
from GVAMP_toolbox import*
from inverse_stieltjes import*
import pandas as pd

In [ ]:
# parameters

asp_vec = np.linspace(0.25,3,50)   # aspect_ratio
delta0 = 0 # noise (only for linear regression)
rho = 0.1   # sparsity of the teacher
sigma = 1  # teacher element-wise variance (Gauss-Bernoulli)
reg = 0.1 # value of regularization
pen = 0     # ratio in the elastic net, 0 is pure l2, 1 is pure l1
niter = 400    # maximum number of iterations
tol = 1e-6     # tolerance on the result
dist=0            # continuous spectrum (0) or row-orthogonal (1)
lo=3          # choosing the problem : lo: 0 - linear regression with noisy linear teacher, 1- linear regression with noiseless sign teacher, 2-linear SVC with noiseless sign teacher, 3-logistic regression with noiseless sign teacher
damp = 0.1      # damp

In [ ]:
E_vec,ang_vec = replica(lo,delta0,rho,sigma,reg,pen,dist,asp_vec,damp,niter,tol)

In [ ]:
plt.plot(asp_vec,ang_vec)

In [ ]:
# Experiment

n_exp = 20
asp_vec_n = np.linspace(0.25,3,n_exp) # aspect ratios 
E_vec_n = np.zeros(n_exp)
ang_vec_n = np.zeros(n_exp)
rho = 0.1  # sparsity parameter of the teacher
delta0 = 0
n_avg = 100 # averaging number
l2 = 1
for i in range(n_exp):
    print((i+1)/n_exp*100,'%')
    count = 0
    asp = asp_vec_n[i]
    if asp == 0.25:
        d = 1000
    else:
        d = 500
    n = int(d*asp)
    #clf = LinearSVC(loss = 'hinge',tol = 0.00001,C=1/l2,max_iter = 100000)
    #clf = linear_model.ElasticNet(alpha=0.001/n,l1_ratio=1,max_iter = 100000,tol = 0.00001)
    clf = linear_model.LogisticRegression(penalty='l1',tol=1e-7,C=1/l2,fit_intercept=False,max_iter=1000000,solver = 'saga')
    while count<n_avg: 
            print(count)
            x0 = gauss_bernoulli(rho,d,sigma)
            w = np.random.normal(0,np.sqrt(delta0),n)
            F_train,D=build_matrix((1-asp)**2,(1+asp)**2,n,d,asp,1) #F_train = np.random.normal(0,np.sqrt(1/n),(n,d)) #F_train,D=build_matrix((1-asp)**2,(1+asp)**2,n,d,asp,1) 
            y_train = np.sign(F_train@x0+w)
            clf.fit(F_train,y_train)
            x_train = clf.coef_
            #x_train = np.linalg.inv(np.transpose(F_train)@F_train+l2*np.eye(d))@np.transpose(F_train)@y_train
            print(np.linalg.norm(x_train))
            if np.linalg.norm(x_train)>1e-9:
                val = x_train@x0/(np.linalg.norm(x0)*np.linalg.norm(x_train))
                ang_vec_n[i] = ang_vec_n[i]+val
                E_vec_n[i] = E_vec_n[i]+np.mean((x0-x_train)**2)
                count = count+1
            
    E_vec_n[i] = 1/n_avg*E_vec_n[i]
    ang_vec_n[i] = np.arccos(1/n_avg*ang_vec_n[i])

In [ ]:
print(ang_vec_n)

In [ ]:
plt.figure()
plt.title("Logistic_regression_Gaussian")
plt.plot(asp_vec,ang_vec,label="SE_prediction_angle")
plt.plot(asp_vec_n,ang_vec_n,'.',label="experiment_angle")
#plt.plot(asp_vec,E_vec,label="SE_prediction_MSE")
#plt.plot(asp_vec_n,E_vec_n,'.',label="experiment_MSE")
plt.ylabel("angle")
plt.xlabel("M/N")
plt.legend()
#plt.savefig('Square_l2_row_orth', dpi=500, quality = 95)

In [ ]:
df = pd.DataFrame(ang_vec, columns=["colummn"])
df.to_csv('test_data2_th.csv', index=False)

In [ ]:
df = pd.DataFrame(ang_vec_n, columns=["colummn"])
df.to_csv('logistic_l1_1_RO_rho_0.1_exp.csv', index=False)